In [4]:
#!pip install pyvisa

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pyvisa
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import csv
%matplotlib qt

In [2]:
def update_plot(fig, ax, line, lstVoltage, lstAbsCurrent):
    line.set_data(lstVoltage, lstAbsCurrent)
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()

INITIALIZE

In [3]:
# Model 2657A
rm = pyvisa.ResourceManager()

In [4]:
# rm = pyvisa.ResourceManager()




# # Keithley 2000
# keith1 = rm.open_resource('GPIB0::24::INSTR')
# keith1.timeout = 100000
# keith1.write('*RST')
# Keithley 2400
keith = rm.open_resource('GPIB0::23::INSTR')
keith.timeout = 100000
keith.write('*RST')
  
# picoamp meter
pico = rm.open_resource('GPIB0::22::INSTR')
pico.timeout = 10000
pico.write('*RST')


6

In [5]:

m2657A = rm.open_resource('TCPIP0::132.181.53.204::inst0::INSTR')
m2657A.timeout = 10000
m2657A.write('*RST')

6

In [6]:
m2657A.write(f"beeper.beep(2,100)")

20

In [7]:
sweepFilePath = "C:\\Users\\gtd19\\Semsem\\bGa2O3\\KeithleyControl\\TSP_2657A\\2657A_DiodeRL\\DiodeRL.tsp"

m2657A.write("loadscript simpleSweep")
with open(sweepFilePath) as fp:
    for line in fp:
        m2657A.write(line)
m2657A.write("endscript")
m2657A.write("simpleSweep.run()")
fp.close()

In [6]:
def setM2657AIVMeasurement (iRange = 0.02, iLimit = 0.02, nplc = 1, delay = 0.05):
    # Reset and initialize instrutment
    m2657A.write(f"reset()")
    m2657A.write(f"status.reset()")
    m2657A.write(f"errorqueue.clear()")
    
    # Configure source function as 2W DCVOLTS
    m2657A.write(f"smua.source.func = smua.OUTPUT_DCVOLTS")
    m2657A.write(f"smua.sense = smua.SENSE_LOCAL")
    m2657A.write(f"display.screen = display.SMUA")
    m2657A.write(f"display.smua.measure.func = display.MEASURE_DCAMPS")
    m2657A.write(f"display.smua.digits = display.DIGITS_5_5")

    #Set up current compliance
    m2657A.write(f"smua.source.limiti = %f" %iLimit)

    #Set up current range 
    #Set up current range
    if iRange == "auto":
        m2657A.write(f"smua.measure.autorangei = smua.AUTORANGE_ON")
    else:
        m2657A.write(f"smua.measure.autorangei = smua.AUTORANGE_OFF")
        m2657A.write(f"smua.measure.rangei = %f" %iRange)

    

    #Set the measurement integration time
    m2657A.write(f"smua.measure.nplc = %f" %nplc)
    m2657A.write(f"smua.measure.delay = %f" %delay)

    

    


    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
	
    m2657A.write(f"beeper.beep(0.5,100)")
    return True
def measureM2657AOnePoint(setV, outputON = False, resetV = True):
    #setM2657AIVMeasurement()
    m2657A.write(f"smua.source.levelv = %f" %setV)
    
    #Configure the reading buffers
    m2657A.write(f"smua.nvbuffer1.clear()")
    m2657A.write(f"smua.nvbuffer1.appendmode = 1")
    m2657A.write(f"smua.nvbuffer1.collecttimestamps = 1")
    m2657A.write(f"smua.nvbuffer1.collectsourcevalues = 0")
    m2657A.write(f"smua.nvbuffer1.fillmode = smua.FILL_ONCE")
    m2657A.write(f"smua.nvbuffer2.clear()")
    m2657A.write(f"smua.nvbuffer2.appendmode = 1")
    m2657A.write(f"smua.nvbuffer2.collecttimestamps = 1")
    m2657A.write(f"smua.nvbuffer2.collectsourcevalues = 0")
    m2657A.write(f"smua.nvbuffer2.fillmode = smua.FILL_ONCE")
    
    
    if (not outputON):
        m2657A.write(f"smua.source.output = 1")
    m2657A.write(f"smua.measure.i(smua.nvbuffer1)")
    m2657A.write(f"smua.measure.v(smua.nvbuffer2)")
    #time.sleep(2)

    #Turn off the output
    if resetV:
        m2657A.write(f"smua.source.levelv = 0")
        m2657A.write(f"smua.source.output = 0")
    
    m2657A.write(f"printbuffer(1, 1, smua.nvbuffer1.readings, smua.nvbuffer2.readings)")
    rawData = m2657A.read()
    # print("%g\t%g\t%g", smua.nvbuffer1.timestamps[1], smua.nvbuffer1.readings[1], smua.nvbuffer2.readings[1])
    arrRawData = rawData.split(",")
    #print ("rawData: ", rawData)

    return [float(arrRawData[i]) for i in range(len(arrRawData))]
setM2657AIVMeasurement(0.02, 0.02, 1, 0.05)
# rur = measureM2657AOnePoint(1,False,True)
# print (results)
current, voltage = measureM2657AOnePoint(1,False,True)
print ("Current: ", current, "Voltage: ", voltage)

Current:  -1.42357e-07 Voltage:  0.99944


In [15]:
def fncMeasureOneM2657A(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='241108\\Dev02\\Test',\
                  askToStart=False,\
                  vRange = 200,\
                  iRange = "auto",\
                  iLimit = 1e-2,\
                  nplc = 1, \
                  delay = 0.5,\
                  breakDownVolt = 3000,\
                  breakDownCurr = 0.02):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    # vRange = 500
    # vILIM = 2e-3
    # iRange = '2e-9'
    #iNPLC = 1
    #vILIM = 'SOUR:VOLT:ILIM' + vILIM
    #iRange = ':SENSE:CURR:RANG ' + iRange



    setM2657AIVMeasurement(iRange, iLimit, nplc, delay)
    absVstart = abs(Vstart)
    absVstop = abs(Vstop)
    # Set up source range
    if abs(Vstart) > abs(Vstop): 
        m2657A.write(f"smua.source.rangev = %f" %absVstart)
    else:
        m2657A.write(f"smua.source.rangev = %f" %absVstop)
    
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Absolute Current (A)')
    ax.set_yscale('log')
    
    

    plt.show()
    time.sleep(1)
    m2657A.write(f"smua.source.output = 1")
    curExceedLimitMeasNum = 0 # number of times the current exceeds the current limit - error
    afterBreakDownMeasNum = 10 # number of additional measurements after the current reaches the current limit at the flat region
    curLimitError = 0.0003
    curLimitLowerBound = breakDownCurr - curLimitError
    # Measurement loop
    for i in range(stepNum):
        
        if np.absolute(Vstart + i*Vstep) > breakDownVolt:
            break
        #pico.write(':INIT:IMM')
        #strVol = '{0:.2f}'.format(i*0.2)
        #strVol = ':SOUR:VOLT {0:.2f}'.format(Vstart + i*Vstep)
        #pico.write(strVol)
        current, voltage = measureM2657AOnePoint(Vstart + i*Vstep,True,False)
        
        # results = pico.query(':MEASure:CURRent?')
        # #print(results)

        # current = results.split(',')[0][:-1]
        #voltage = l
        lstVoltage.append(voltage)
        lstCurrent.append(current)
        lstAbsCurrent.append(abs(current))
        update_plot(fig, ax, line,lstVoltage, lstAbsCurrent)
        if (abs((current)) >  breakDownCurr): #or (stop_flag == True)
            break
        if (abs(current) >  curLimitLowerBound):
            curExceedLimitMeasNum+=1
        if (curExceedLimitMeasNum >= afterBreakDownMeasNum):
            break
        time.sleep(0.1)
    #Turn off the output
    m2657A.write(f"smua.source.levelv = 0")
    m2657A.write(f"smua.source.output = 0")

    pico.write(':SOUR:VOLT:STAT OFF')
    dataToWrite = [['vRange: ', vRange]]
    dataToWrite.append(['iLimit: ', iLimit])
    dataToWrite.append(['iRange: ', iRange[16:]])
    dataToWrite.append(['nplc: ', nplc, 'delay: ', delay])
    dataToWrite.append(['data Number:', stepNum])
    dataToWrite.append(['V_meas', 'I','abs_I'])
    # lstCalVoltage = [(Vstart + i*Vstep) for i in range(len(lstAbsCurrent))]

    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '.png')

In [43]:
fncMeasureOneM2657A([0, 1, 0.1], sample, 1, folderPath)

241112_Test\Dev01\Air\Sweep1AirStar0_Stop1_Step0.1.csv


In [8]:
def fncMeasureOnePico(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = 500
    vILIM = 2e-3
    iRange = '2e-9'
    iNPLC = 1
    #vILIM = 'SOUR:VOLT:ILIM' + vILIM
    iRange = ':SENSE:CURR:RANG ' + iRange
    breakDownVolt = 505
    breakDownCurr = 2.5e-3
    pico.write('*RST')
    pico.write(iRange)
    pico.write("SOUR:VOLT:RANG %f" % vRange)
    pico.write("SOUR:VOLT:ILIM %f" %vILIM)
    pico.write(":SENSE:CURR:NPLC %f" %iNPLC )
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    data = [['Volt', 'Curr'],['V', 'A']]
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Absolute Current (A)')
    ax.set_yscale('log')
    
    
#     messStartMeasurement = 'File Paht: ' + filePath + '.\n' + " Do you want to start the measurement?" 
#     # Display a message box and ask for the operator's choice
#     choice = messagebox.askquestion("Measurement", messStartMeasurement)
    
#     if choice == 'yes':
    plt.show()
    time.sleep(1)
    pico.write(':SOUR:VOLT:STAT ON')
    curExceedLimitMeasNum = 0 # number of times the current exceeds the current limit - error
    afterBreakDownMeasNum = 10 # number of additional measurements after the current reaches the current limit at the flat region
    curLimitError = 0.0003
    curLimitLowerBound = breakDownCurr - curLimitError
    # Measurement loop
    for i in range(stepNum):
        lstVoltage.append(Vstart + i*Vstep)
        if np.absolute(Vstart + i*Vstep) > breakDownVolt:
            break
        pico.write(':INIT:IMM')
        #strVol = '{0:.2f}'.format(i*0.2)
        strVol = ':SOUR:VOLT {0:.2f}'.format(Vstart + i*Vstep)
        pico.write(strVol)
        results = pico.query(':MEASure:CURRent?')
        #print(results)
        current = results.split(',')[0][:-1]
        #voltage = l
        lstCurrent.append(float(np.squeeze(current)))
        lstAbsCurrent.append(np.abs(float(np.squeeze(current))))
        update_plot(fig, ax, line,lstVoltage, lstAbsCurrent)
        if (abs(float(np.squeeze(current))) >  breakDownCurr): #or (stop_flag == True)
            break
        if (abs(float(np.squeeze(current))) >  curLimitLowerBound):
            curExceedLimitMeasNum+=1
        if (curExceedLimitMeasNum >= afterBreakDownMeasNum):
            break
        
        
        time.sleep(0.1)
    pico.write(':SOUR:VOLT:STAT OFF')
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['vILIMIT', vILIM])
    dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite.append(['V_cal', 'I','abs_I'])
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(len(lstAbsCurrent))]

    for i in range(len(lstCurrent)):
        dataToWrite.append([round(Vstart + i * Vstep, 2), lstCurrent[i], lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '.png')

In [9]:
def fncMeasureOneKL1(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = "AUTO"
    iCompliance = 1
    iNPLC = 1
    delay = 0.1
    keith1.write('*RST')
    keith1.write(':ROUT:TERM REAR')
    keith1.write(':SOUR:FUNC VOLT')
    keith1.write(':SENS:FUNC "CURR:DC"')
    keith1.write(":SOUR:VOLT:STAR %0.2f" % Vstart)
    keith1.write(":SOUR:VOLT:STOP %0.2f" % Vstop)
    keith1.write(":SOUR:VOLT:STEP %0.3f" % Vstep)
    vRange1 = ':SOUR:SWE:RANG ' + vRange
    keith1.write(vRange)
    iCompliance = ':SENSE:CURR:PROT '+ str(iCompliance)
    keith1.write(iCompliance)
    keith1.write(":SOUR:VOLT:MODE SWE")
    keith1.write(":SENSE:CURR:PROT 1" )
    keith1.write(':SENSE:CURR:NPLC %f' %iNPLC )
    keith1.write(':SOUR:SWE:SPAC LIN')
    keith1.write("TRAC:CLE")
    keith1.write('TRAC:POIN %f' %stepNum)
    keith1.write(':FORM:ELEM:SENS VOLT,CURR')
    keith1.write(':TRIG:COUN %f' % stepNum)
    keith1.write(':SOUR:SWE:POIN %f' %stepNum)
    keith1.write(':SOUR:DEL %0.2f' % delay)
    keith1.write(':OUTP ON')
    keith1.write(":INIT")
    keith1.write('*WAI')
    keith1.write(':OUTP OFF')

    time.sleep(1)

    data = keith1.query_ascii_values("FETC?")
    #print(data)
    # Read data from buffer of Keithley 2400 and write to the file
    lstCurrent = data[1::2] # get all odd elements, which are currents

    lstVoltage = data[0::2] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    #plt.xticks(fontsize = 18)
    ax.set_ylabel('Current (A)')
    ax.set_yscale('log')
    #plt.yticks(fontsize = 18)
    # Function to update plot
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '.png')


In [10]:
def fncMeasureOneKL(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = "AUTO"
    iCompliance = 1
    iNPLC = 1
    delay = 0.1
    keith.write('*RST')
    keith.write(':ROUT:TERM REAR')
    keith.write(':SOUR:FUNC VOLT')
    keith.write(':SENS:FUNC "CURR:DC"')
    keith.write(":SOUR:VOLT:STAR %0.2f" % Vstart)
    keith.write(":SOUR:VOLT:STOP %0.2f" % Vstop)
    keith.write(":SOUR:VOLT:STEP %0.3f" % Vstep)
    vRange = ':SOUR:SWE:RANG ' + vRange
    keith.write(vRange)
    iCompliance = ':SENSE:CURR:PROT '+ str(iCompliance)
    keith.write(iCompliance)
    keith.write(":SOUR:VOLT:MODE SWE")
    keith.write(":SENSE:CURR:PROT 1" )
    keith.write(':SENSE:CURR:NPLC %f' %iNPLC )
    keith.write(':SOUR:SWE:SPAC LIN')
    keith.write("TRAC:CLE")
    keith.write('TRAC:POIN %f' %stepNum)
    keith.write(':FORM:ELEM:SENS VOLT,CURR')
    keith.write(':TRIG:COUN %f' % stepNum)
    keith.write(':SOUR:SWE:POIN %f' %stepNum)
    keith.write(':SOUR:DEL %0.2f' % delay)
    keith.write(':OUTP ON')
    keith.write(":INIT")
    keith.write('*WAI')
    keith.write(':OUTP OFF')

    time.sleep(1)

    data = keith.query_ascii_values("FETC?")
    #print(data)
    # Read data from buffer of Keithley 2400 and write to the file
    lstCurrent = data[1::2] # get all odd elements, which are currents

    lstVoltage = data[0::2] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    #plt.xticks(fontsize = 18)
    ax.set_ylabel('Current (A)')
    ax.set_yscale('log')
    #plt.yticks(fontsize = 18)
    # Function to update plot
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '.png')


In [16]:
def fncMeasureOne(arrVMeasurement, \
                  device= 'pico',\
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False):
    if (device == 'pico'):
        fncMeasureOnePico(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'keith'):
        fncMeasureOneKL(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'keith1'):
        fncMeasureOneKL1(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2657A'):
        fncMeasureOneM2657A(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    else:
        print("Incorrect device name")

In [17]:
def fncMeasureMultiple(settings =['pico', [[0, 0.5, 0.1]]], \
                  Sample = 'Test',\
                  startNum = 1, \
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False): 
    sweepNum = 0
    dev, arrVMeasurements = settings
    for j in range (len(arrVMeasurements)):
        sweepNum = j + startNum
        fncMeasureOne(arrVMeasurements[j], dev , Sample, sweepNum, folderPath)
        time.sleep(5)

In [18]:
folderPath = '241112_Ni'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
    
folderPath = folderPath+ '\\D2m1m'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")

folderPath = folderPath+ '\\Was'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
sample = folderPath[-3:]

Folder already exists
Folder already exists
Folder already exists


In [53]:
#run Keithley 2400 [0, 3, 0.01],\
arrKLVMeasMulti =["keith",[[0, 3, 0.01],\
                            #[0, 3, 0.01],\
                            [0, -3, -0.01]]]
fncMeasureMultiple(arrKLVMeasMulti,sample, 1, folderPath)

240805_Fab240727_NiNiO_NegLOnBrassSM\Dev20\J01\Sweep1J01Star0_Stop3_Step0.01.csv
240805_Fab240727_NiNiO_NegLOnBrassSM\Dev20\J01\Sweep2J01Star0_StopM3_StepM0.01.csv


In [49]:
#run picoamp meter
arrPicoVMeasMulti =["pico", [[0, 1.5, 0.01],\
                [0, -1.5, -0.01],\
                [0, -500, -1],\
                 [0, 1.5, 0.01],\
                 [0, -1.5, -0.01]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 3, folderPath)

241112_Ni\D2m1m\H01\Sweep3H01Star0_Stop1.5_Step0.01.csv


KeyboardInterrupt: 

In [19]:
#run picoamp meter
arrPicoVMeasMulti =["M2657A", [[0, 3000, 100]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 1, folderPath)

241112_Ni\D2m1m\Was\Sweep1WasStar0_Stop3000_Step100.csv


In [50]:
#run picoamp meter
arrPicoVMeasMulti =["M2657A", [[0, -1.5, -0.1],\
                [0, 1.5, 0.1],\
                [0, 500, 1],\
                 [0, -1.5, -0.1],\
                 [0, 1.5, 0.1]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 3, folderPath)

241112_Ni\D2m1m\H01\Sweep3H01Star0_StopM1.5_StepM0.1.csv
241112_Ni\D2m1m\H01\Sweep4H01Star0_Stop1.5_Step0.1.csv
241112_Ni\D2m1m\H01\Sweep5H01Star0_Stop500_Step1.csv
241112_Ni\D2m1m\H01\Sweep6H01Star0_StopM1.5_StepM0.1.csv
241112_Ni\D2m1m\H01\Sweep7H01Star0_Stop1.5_Step0.1.csv


In [28]:
#run Keithley 2400
arrKLVMeasMulti =["keith", [[0, 3, 0.01],\
                [0, -3, -0.01]]]
fncMeasureMultiple(arrKLVMeasMulti,sample, 8, folderPath)

240805_Fab240727_NiNiO_NegLOnBrassSM\Dev05\D04\Sweep8D04Star0_Stop3_Step0.01.csv
240805_Fab240727_NiNiO_NegLOnBrassSM\Dev05\D04\Sweep9D04Star0_StopM3_StepM0.01.csv


In [27]:
arrVMeas = [0, -3, -0.01]
fncMeasureOne(arrVMeas, sample, 1, folderPath)

Incorrect device name


In [42]:
#run picoamp meter
arrPicoVMeasMulti =["keith", [[0, 3, 0.01]]]#,\
                #[0, -1.5, -0.01],\
                #[0, -500, -1],\
                 #[0, 1.5, 0.01],\
                 #[0, -1.5, -0.01]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 8, folderPath)

231210_Fab231209_3DPrintedSM\Dev16\F01\Sweep8F01Star0_Stop3_Step0.01.csv


In [5]:
#run Keithley 2400 [0, 3, 0.01],\
arrKLVMeasMulti =["keith1",[[0, 3, 0.01],\
                            [0, 3, 0.01],\
                            [0, -3, -0.01]]]
fncMeasureMultiple(arrKLVMeasMulti,sample, 0, folderPath)

NameError: name 'fncMeasureMultiple' is not defined

In [55]:
#run picoamp meter
arrPicoVMeasMulti =["keith", [[0, 3, 0.01]]]#,\
                #[0, -1.5, -0.01],\
                #[0, 3, 0.01],\
                 #[0, -500, -1],\
                 #[0, -1.5, -0.01]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 4, folderPath)

240805_Fab240727_NiNiO_NegLOnBrassSM\Dev20\J01\Sweep4J01Star0_Stop3_Step0.01.csv
